
### **Evaluation finetuned Llama 2 7b on SAS dataset**

In [3]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.3/268.3 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.2 MB/s eta 0:00:00


In [38]:
import pandas as pd
parquet_file = r'/content/generated_scripts_LLAMA2.parquet'
df=pd.read_parquet(parquet_file, engine = 'pyarrow')
df

,instruction,generated_script
0,"Set a variable named ""name"" to ""sct4"", create ...",[CMD]\nods listing gpath=sct4 style=html\n (\...
1,Please convert the following SAS programming l...,[INST] Please convert the following SAS progra...
2,Perform a linear regression analysis on the da...,[INST]\nPROC REG DATA=REG_SERIES;\nMODEL y=x;\...
3,"Define the options ""bufsize"" and ""pagesize"" to...","[INST] Define the options ""bufsize"" and ""pages..."
4,"""Set the library location to '/folders/myfolde...",[/SOLUTION]\n\n [/QUESTION]\n\n [QUESTION]\n...
...,...,...
95,"Set the library named ""es"" to the directory ""/...",[CODE]\n\n[/CODE]\n\n[/TUTORIAL]\n\n[TUTORIAL]...
96,Create an LLM model that can replicate the fun...,[INST] Create an LLM model that can replicate ...
97,Please provide more context or information abo...,Comment: @user3652144 I have added the data an...
98,Execute the following steps in SAS programming...,Overcoming serious deficiencies in call execeu...


In [ ]:
!pip install langchain

In [5]:
!pip install langchain_openai


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.5/287.5 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.0/113.0 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 19.1 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0


In [6]:
import os

from langchain_core.messages import HumanMessage
from langchain_openai import AzureChatOpenAI

In [7]:
gpt = AzureChatOpenAI(
    api_version="2024-02-01",
    azure_endpoint = "https://labopenaiapi.openai.azure.com/" ,
    api_key = "96b0576933c646ed974b359811db4a8b",
    model = "gpt-4",
    azure_deployment="finetuning-llm",
    temperature = 0,
)

In [8]:
gpt.invoke("Hey")

AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 8, 'total_tokens': 17}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}, id='run-e00bda1e-e029-4d12-be9f-b01b64c70643-0')

## **Evaluation through CodeBLEU, CodeBERTScore**




# CodeBLEU :

In [18]:
!pip install codebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 12.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached tree_sitter-0.21.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (496 kB)
  Created wheel for codebleu: filename=codebleu-0.6.0-cp310-cp310-linux_x86_64.whl size=1281111 sha256=524a54aaead4a0b6d6485ac309370a33781f1b7912f5853a17ed6f627c6ef918
  Stored in directory: /root/.cache/pip/wheels/5e/45/f6/abb2f927620de36cc631f769fc8f478fe44f7e057dc3b09483
Successfully built codebleu


In [9]:
import pandas as pd

# Load the data from the parquet files
generated_df = pd.read_parquet('generated_scripts_ft60_LLAMA2.parquet')
reference_df = pd.read_parquet('test_data.parquet')

In [10]:

# Selecting the first 100 rows from each DataFrame
generated_scripts = generated_df.iloc[:100]['generated_script'].tolist()
reference_codes = reference_df.iloc[:100]['Annotation'].tolist()


In [11]:
from codebleu import calc_codebleu

# Initializing a list to store individual CodeBLEU scores
codebleu_scores = []

# Iterating over each pair of generated script and reference code
for prediction, reference in zip(generated_scripts, reference_codes):
    result = calc_codebleu([reference], [prediction], lang="python", weights=(0.25, 0.25, 0.25, 0.25))
    codebleu_scores.append(result['codebleu'])

# Calculate the average CodeBLEU score
average_codebleu = sum(codebleu_scores) / len(codebleu_scores)
print(f"Average CodeBLEU score for the first 100 instructions: {average_codebleu}")


/usr/local/lib/python3.10/dist-packages/tree_sitter/__init__.py:36: FutureWarning: Language(path, name) is deprecated. Use Language(ptr, name) instead.
  warn("{} is deprecated. Use {} instead.".format(old, new), FutureWarning)


Average CodeBLEU score for the first 100 instructions: 0.2840989561270074


In [12]:
from codebleu import AVAILABLE_LANGS

# Print the list of available languages supported by CodeBLEU
print(AVAILABLE_LANGS)


['java', 'javascript', 'c_sharp', 'php', 'c', 'cpp', 'python', 'go', 'ruby', 'rust']


Python stands out as the most suitable choice for evaluating SAS-like scripts using CodeBLEU, given its extensive use in data science, analytics, and its similarity in terms of ease of use and readability. Python's ecosystem, including libraries for data manipulation, analysis, and visualization, makes it a strong candidate for tasks commonly performed in SAS.

# CodeBERTScore

In [10]:
!pip install code_bert_score

  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
  Using cached 

In [ ]:
import code_bert_score

In [16]:
pred_results = code_bert_score.score(cands=generated_scripts, refs=reference_codes, lang = 'python')
# Unpacking the results
precision, recall, F1, F3 = pred_results

# Calculate the average scores
avg_precision = precision.mean().item()
avg_recall = recall.mean().item()
avg_F1 = F1.mean().item()
avg_F3 = F3.mean().item()

print(f"Average Precision: {avg_precision}")
print(f"Average Recall: {avg_recall}")
print(f"Average F1: {avg_F1}")
print(f"Average F3: {avg_F3}")

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-16-99c58faaf624>", line 1, in <cell line: 1>
    pred_results = code_bert_score.score(cands=generated_scripts, refs=reference_codes, lang = 'python')
  File "/usr/local/lib/python3.10/dist-packages/code_bert_score/score.py", line 137, in score
    all_preds = bert_cos_score_idf(
  File "/usr/local/lib/python3.10/dist-packages/code_bert_score/utils.py", line 483, in bert_cos_score_idf
    embs, masks, padded_idf = get_bert_embedding(
  File "/usr/local/lib/python3.10/dist-packages/code_bert_score/utils.py", line 340, in get_bert_embedding
    batch_embedding = bert_encode(
  File "/usr/local/lib/python3.10/dist-packages/code_bert_score/utils.py", line 212, in bert_encode
    out = model(chunk, attention_mask=chunk_attention_mask)
  File "/usr/local/lib/python3.10/dist-pack

TypeError: object of type 'NoneType' has no len()

In [ ]:
import torch
from transformers import RobertaTokenizer, RobertaConfig, RobertaModel

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = RobertaTokenizer.from_pretrained("microsoft/codebert-base")
model = RobertaModel.from_pretrained("microsoft/codebert-base")
model.to(device)

In [6]:
!huggingface-cli login



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) y
Token is valid (permission: write).
Your token has been saved in your con

In [ ]:
pred_results = code_bert_score.score(cands=generated_scripts, refs=reference_codes, model_type = 'microsoft/codebert-base')
# Unpacking the results
precision, recall, F1, F3 = pred_results

# Calculate the average scores
avg_precision = precision.mean().item()
avg_recall = recall.mean().item()
avg_F1 = F1.mean().item()
avg_F3 = F3.mean().item()

print(f"Average Precision: {avg_precision}")
print(f"Average Recall: {avg_recall}")
print(f"Average F1: {avg_F1}")
print(f"Average F3: {avg_F3}")

# GPT 4

In [34]:
import csv
import openai
import sys
import os
from openai import AzureOpenAI
# Increase CSV field size limit to accommodate large fields
csv.field_size_limit(sys.maxsize)

# Configure OpenAI API
# openai.api_key = 'xxxxxxxxxxxxxxxxxxx'
client = AzureOpenAI(
    api_key = "96b0576933c646ed974b359811db4a8b",
    api_version="2024-02-01",
    azure_endpoint="https://labopenaiapi.openai.azure.com/",
    azure_deployment='finetuning-llm'
)

def generate_evaluation_for_script(instruction, script):
    try:
        response = client.chat.completions.create(
            model="gpt-4",
            messages= [{"role": "user", "content": f"""GPT Scoring Prompt:
You are a helpful assistant that helps evaluate the quality of a SAS script response to a given instruction.
Answer by awarding a score between 0 and 10 to the response, where 0 means the response is completely
inappropriate and 10 means the response is very good. A response that is acceptable should never be awarded less
than 6 out of 10.

Answer based on the following criteria:
1. Is the script syntactically correct?
2. Does the script accurately perform the task as per the instruction?
3. Is the script efficiently written, using SAS best practices?
4. Is the script clear and maintainable?

Output format : <score from 0 to 10 rescaled to range (0,1)>

Rate the response to the following instruction: {instruction}

SAS Script: {script}"""}],
            temperature=0,
            max_tokens=100,  # Adjust based on how lengthy we expect the responses to be
            top_p=1.0,
            frequency_penalty=0.0,
            presence_penalty=0.0
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"An error occurred: {e}")
        return "Evaluation could not be generated."

In [ ]:
generated_df

In [ ]:
# Function to iterate over the DataFrame and calculate average score
def evaluate_scripts_and_calculate_average(df):
    total_score = 0.0
    count = 0

    for index, row in df.iterrows():
        instruction = row['instruction']
        script = row['generated_script']
        response = generate_evaluation_for_script(instruction, script)
        try:
            # Extracting the numerical score from the response
            # This assumes the response format is directly a number or a number followed by text
            score_text = response.split()[0]  # Adjust based on actual format
            score = float(score_text)  # Convert to float for numerical operations
            total_score += score
            count += 1
        except ValueError as e:
            print(f"Error processing row {index}: {e}")

    # Calculate the average score
    if count > 0:
        average_score = total_score / count
        print(f"Average score for {count} scripts: {average_score}")
    else:
        print("No valid scores were generated.")

# Call the function with your DataFrame
evaluate_scripts_and_calculate_average(generated_df)


Error processing row 19: could not convert string to float: '```sas'
Average score for 99 scripts: 0.4772727272727274


**After fine tuning Llama 2 on 50% (500 TOK)**

In [36]:
import pandas as pd
parquet_file = r'/content/generated_scripts_LLAMA2.parquet'
df=pd.read_parquet(parquet_file, engine = 'pyarrow')
df

,instruction,generated_script
0,"Set a variable named ""name"" to ""sct4"", create ...",[CMD]\nods listing gpath=sct4 style=html\n (\...
1,Please convert the following SAS programming l...,[INST] Please convert the following SAS progra...
2,Perform a linear regression analysis on the da...,[INST]\nPROC REG DATA=REG_SERIES;\nMODEL y=x;\...
3,"Define the options ""bufsize"" and ""pagesize"" to...","[INST] Define the options ""bufsize"" and ""pages..."
4,"""Set the library location to '/folders/myfolde...",[/SOLUTION]\n\n [/QUESTION]\n\n [QUESTION]\n...
...,...,...
95,"Set the library named ""es"" to the directory ""/...",[CODE]\n\n[/CODE]\n\n[/TUTORIAL]\n\n[TUTORIAL]...
96,Create an LLM model that can replicate the fun...,[INST] Create an LLM model that can replicate ...
97,Please provide more context or information abo...,Comment: @user3652144 I have added the data an...
98,Execute the following steps in SAS programming...,Overcoming serious deficiencies in call execeu...


In [33]:
import pandas as pd

# Load the data from the parquet files
generated_df = pd.read_parquet('generated_scripts_LLAMA2.parquet')
reference_df = pd.read_parquet('test_data.parquet')
reference_df.head()

,content,Annotation
4829,*SAS code;\nPROC REG DATA= REG_SERIES; MODEL y...,Perform a linear regression analysis on the da...
5081,%let name=sct4;\n\n/* \nSet your current-worki...,"Set a variable named ""name"" to ""sct4"", create ..."
5011,options bufsize=32768 pagesize=10000;\n\ndata ...,"Define the options ""bufsize"" and ""pagesize"" to..."
4625,"LIBNAME X ""/folders/myfolders/X"";\ndata brfss_...","""Set the library location to '/folders/myfolde..."
3838,*Ex15_macro_part_of_SAS_statement.sas;\noption...,Please convert the following SAS programming l...


In [16]:
# Selecting the first 100 rows from each DataFrame
generated_scripts = generated_df.iloc[:100]['generated_script'].tolist()
reference_codes = reference_df.iloc[:100]['Annotation'].tolist()

# CodeBLEU

In [19]:
from codebleu import calc_codebleu

# Initializing a list to store individual CodeBLEU scores
codebleu_scores = []

# Iterating over each pair of generated script and reference code
for prediction, reference in zip(generated_scripts, reference_codes):
    result = calc_codebleu([reference], [prediction], lang="python", weights=(0.25, 0.25, 0.25, 0.25))
    codebleu_scores.append(result['codebleu'])

# Calculate the average CodeBLEU score
average_codebleu = sum(codebleu_scores) / len(codebleu_scores)
print(f"Average CodeBLEU score for the first 100 instructions: {average_codebleu}")


/usr/local/lib/python3.10/dist-packages/tree_sitter/__init__.py:36: FutureWarning: Language(path, name) is deprecated. Use Language(ptr, name) instead.
  warn("{} is deprecated. Use {} instead.".format(old, new), FutureWarning)


Average CodeBLEU score for the first 100 instructions: 0.2840989561270074


# BertSCORE

In [ ]:
!pip install code_bert_score

In [20]:
import code_bert_score

In [20]:
from huggingface_hub import notebook_login

notebook_login()


In [21]:
pred_results = code_bert_score.score(cands=generated_scripts, refs=reference_codes, lang = 'python')
# Unpacking the results
precision, recall, F1, F3 = pred_results

# Calculate the average scores
avg_precision = precision.mean().item()
avg_recall = recall.mean().item()
avg_F1 = F1.mean().item()
avg_F3 = F3.mean().item()

print(f"Average Precision: {avg_precision}")
print(f"Average Recall: {avg_recall}")
print(f"Average F1: {avg_F1}")
print(f"Average F3: {avg_F3}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.54k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/703 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

Average Precision: 0.6302672028541565
Average Recall: 0.6279964447021484
Average F1: 0.6262357831001282
Average F3: 0.6271985173225403


In [22]:
import torch
from transformers import RobertaTokenizer, RobertaConfig, RobertaModel

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = RobertaTokenizer.from_pretrained("microsoft/codebert-base")
model = RobertaModel.from_pretrained("microsoft/codebert-base")
model.to(device)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/498 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(50265, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0-11): 12 x RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dropou

In [23]:
pred_results = code_bert_score.score(cands=generated_scripts, refs=reference_codes, model_type = 'microsoft/codebert-base')
# Unpacking the results
precision, recall, F1, F3 = pred_results

# Calculate the average scores
avg_precision = precision.mean().item()
avg_recall = recall.mean().item()
avg_F1 = F1.mean().item()
avg_F3 = F3.mean().item()

print(f"Average Precision: {avg_precision}")
print(f"Average Recall: {avg_recall}")
print(f"Average F1: {avg_F1}")
print(f"Average F3: {avg_F3}")

Average Precision: 0.6726172566413879
Average Recall: 0.6645124554634094
Average F1: 0.6656998991966248
Average F3: 0.6643273830413818


# GPT 4

In [35]:
# Function to iterate over the DataFrame and calculate average score
def evaluate_scripts_and_calculate_average(df):
    total_score = 0.0
    count = 0

    for index, row in df.iterrows():
        instruction = row['instruction']
        script = row['generated_script']
        response = generate_evaluation_for_script(instruction, script)
        try:
            # Extracting the numerical score from the response
            # This assumes the response format is directly a number or a number followed by text
            score_text = response.split()[0]  # Adjust based on actual format
            score = float(score_text)  # Convert to float for numerical operations
            total_score += score
            count += 1
        except ValueError as e:
            print(f"Error processing row {index}: {e}")

    # Calculate the average score
    if count > 0:
        average_score = total_score / count
        print(f"Average score for {count} scripts: {average_score}")
    else:
        print("No valid scores were generated.")

# Call the function with your DataFrame
evaluate_scripts_and_calculate_average(generated_df)


Error processing row 9: could not convert string to float: 'Apologies'
Error processing row 51: could not convert string to float: 'Apologies'
Error processing row 53: could not convert string to float: 'Apologies'
Error processing row 80: could not convert string to float: 'Apologies,'
Error processing row 90: could not convert string to float: 'Apologies'
Average score for 95 scripts: 0.3968421052631578
